In [1]:
from clustering_driver import *

In [2]:
f = "../../../cryoem/Ensemble-reweighting-using-Cryo-EM-particles/processed_images/images_from_structures.npy"
# N = dataloader(f)
params = {"filename": f, "k": 3, "R": 30, "C": 45}
nl, dl = hierarchify_wrapper(f, 3, 30, 45)

Loaded Array:
(256, 128, 128)
starting search tree construction:
	N = 256
	D = 16384
	k = 3
	R = 30
Tree build time::	2.3202351970012387
building data reference list...
list build time::	0.030019040999832214
total:	2.350254238001071


In [4]:
g = "../../../cryoem/Ensemble-reweighting-using-Cryo-EM-particles/processed_images/images_from_structures.npy"
N = dataloader(g)
np.random.shuffle(N)

Loaded Array:
(256, 128, 128)


In [3]:
node_list = nl
data_list = dl

# m_list = [i for i in range(len(N))]
# st_idxs, st_dss = search_tree_associations(node_list, data_list, N)
# ap_idxs, ap_dss = all_pairs_associations(data_list, N)

In [38]:
comp_dict = {}
for m in m_list:
    comp_dict[m] = {"Tree": {}, "Naive": {}}

In [6]:
comp_dict = {}
for m in range(len(data_list)):
    comp_dict[m] = {
        "Tree": {"Index": [], "Distance": []},
        "Naive": {"Index": [], "Distance": []},
    }
for i, a in enumerate(st_idxs):
    comp_dict[a]["Tree"]["Index"].append(f"T{int(i)}")
    comp_dict[a]["Tree"]["Distance"].append(float(st_dss[i]))

for i, a in enumerate(ap_idxs):
    comp_dict[a]["Naive"]["Index"].append(f"N{int(i)}")
    comp_dict[a]["Naive"]["Distance"].append(float(ap_dss[i]))

comp_dict = {"Nodes": [{i: v} for i, v in comp_dict.items()]}

In [20]:
import json
import pickle

tree_dict = {
    "parameters": params,
    "resources": {"node_vals": "tree_node_vals.npy", "data_list": "tree_data_list.npy"},
    "root_node": {},
}

tnl = []
node_vals = []
for i, node in enumerate(node_list):
    if i == 0:
        # tnl.append()
        tree_dict["root_node"] = {
            "node_id": i,
            "node_val_idx": None,
            "children": [c for c in node.children],
            "data_refs": None,
        }
        continue
    if node.data_refs is not None:
        tnl.append(
            {
                "node_id": i,
                "node_val_idx": len(node_vals),
                "children": None,
                "data_refs": [didx for didx in node.data_refs],
            }
        )
        node_vals.append(np.array(data_list[node.val_idx]))
    else:
        tnl.append(
            {
                "node_id": i,
                "node_val_idx": len(node_vals),
                "children": [c for c in node.children],
                "data_refs": None,
            }
        )
        node_vals.append(np.array(node.val))

print(node_vals)
# tree_dict["data_list"] = pickle.dumps(np.array(data_list))
tree_dict["node_list"] = tnl
# serializable_node_vals = pickle.dumps(np.array(node_vals))
# serializable_data_list = pickle.dumps(np.array(data_list))
f = open("tree_hierarchy.json", "w")
f.write(json.dumps(tree_dict, indent=2))
f.close()

np.save("tree_node_vals.npy", np.array(node_vals))
np.save("tree_data_list.npy", np.array(data_list))

[array([[-1.09987369e-12, -1.13595151e-12,  2.31158241e-14, ...,
         1.86082174e-12,  4.47562357e-13, -5.90713535e-13],
       [-6.89544917e-13, -1.06135927e-12, -1.03193571e-12, ...,
         2.37655856e-12,  2.08516995e-12,  3.86516010e-13],
       [ 2.94465038e-13, -6.82638528e-13, -9.22221413e-13, ...,
        -7.99985091e-13,  2.72319468e-12,  2.14921085e-12],
       ...,
       [ 2.47334806e-12,  4.04530016e-12, -1.81202506e-13, ...,
        -7.04810011e-13, -1.26352349e-12, -3.42870422e-13],
       [-1.13265882e-13,  2.62469004e-12,  3.87776936e-12, ...,
        -2.22852953e-13, -8.97391128e-13, -1.24332047e-12],
       [-1.19850791e-12,  5.37013556e-15,  2.57585926e-12, ...,
         4.93101314e-13, -4.25742682e-13, -1.02970875e-12]]), array([[-5.00475706e-13, -1.00135271e-12,  4.11300940e-13, ...,
        -3.93589385e-13,  4.28028628e-13,  3.69915887e-13],
       [-7.32296705e-14, -6.80487306e-13,  1.14013203e-13, ...,
        -1.15676534e-12, -1.43909202e-13,  4.33741144

In [19]:
arr = np.load("tree_node_vals.npy")
arr.shape

(66, 128, 128)

In [28]:
f = open("file.txt", "w")
for element, methods in comp_dict.items():
    f.write(f"Element {element}:\n")
    for method, data in methods.items():
        f.write(f"  {method} Method:\n")
        indices = data["Index"]
        distances = data["Distance"]
        for index, distance in zip(indices, distances):
            f.write(f"    Index: {index}, Distance: {distance}\n")

f.close()

In [38]:
import json

f = open("file.json", "w")
f.write(json.dumps({"Nodes": [{i: v} for i, v in comp_dict.items()]}, indent=2))
f.close()

256


In [8]:
data = []
for N in comp_dict["Nodes"]:
    for element, details in N.items():
        key = element
        tree_neighbors = [
            (
                "green",
                f"Data{element}",
                details["Tree"]["Index"][i],
                details["Tree"]["Distance"][i],
            )
            for i in range(len(details["Tree"]["Index"]))
        ]
        for t in tree_neighbors:
            data.append(t)
        naive_neighbors = [
            (
                "yellow",
                f"Data{element}",
                details["Naive"]["Index"][i],
                details["Naive"]["Distance"][i],
            )
            for i in range(len(details["Naive"]["Index"]))
        ]
        for t in naive_neighbors:
            data.append(t)

for i, node in enumerate(node_list[1:]):
    if node.data_refs is not None:
        for idx in node.data_refs:
            data.append(
                (
                    "purple",
                    f"Node{i+1}",
                    f"Data{idx}",
                    np.linalg.norm(data_list[node.val_idx] - data_list[idx]),
                )
            )
    else:
        for c in node.children:
            data.append(
                (
                    "blue",
                    f"Node{i+1}",
                    f"Node{c}",
                    np.linalg.norm(node.val - node_list[c].val),
                )
            )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


In [11]:
# print(comp_dict)
import json

f = open("sample.json", "w")
f.write(json.dumps(comp_dict, indent=2))
f.close()

In [7]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()
node_colors = {}
for color, A, B, _ in data:
    if color == "blue":
        node_colors[A] = color
    if color == "purple":
        node_colors[A] = color
        node_colors[B] = color
    else:
        node_colors[B] = color
        # node_colors[A] = color
    # node_colors[A] = "blue"
# Add nodes and edges based on relationships
for color, A, B, distance in data:
    G.add_edge(A, B, weight=distance, color=color)
nx.set_node_attributes(G, node_colors, "color")
# Save the graph in GraphML format
nx.write_graphml(G, "graph_representation.graphml")